In [1]:
%matplotlib qt
import torch.nn.functional as F
import torchvision.models as models
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import torch
from torch.autograd import Variable
from torchvision import transforms, datasets

from data_utils_NerveNet import SegmentationData
from data_utils_SegNet import SegmentationData as SegData
import transform_utils_NerveNet as tu

import get_model_stats as gs

%load_ext autoreload
%autoreload 2


In [2]:
img_files = 'image_files.txt' 
num_lines = sum(1 for line in open(img_files, 'r'))
num_lines = sum(1 for line in open(img_files, 'r'))
nums = {'num_train' : int(0.6*num_lines)+2,
        'num_val'   : int(0.2*num_lines),
        'num_test' : int(0.2*num_lines)}

print(nums, num_lines)
print(np.sum(list(nums.values())) == num_lines)

{'num_train': 1278, 'num_val': 425, 'num_test': 425} 2128
True


In [3]:
input_dim = (1, 128, 128)
model_names = ['NerveNet', 'NerveNet_binary', 'SegNet', 'SegNet_binary', 'NerveNet_binary_iGAN']

#model_path = 'models/' + model_name + '.model'
#model = torch.load(model_path, map_location=lambda storage, loc: storage)


In [ ]:
input_dim = (1, 128, 128)
test_transforms = tu.Compose([tu.Resize(input_dim[1::]),
                               tu.ToTensor()])
for mname in model_names:
    model_path = 'models/' + mname + '.model'
    model = torch.load(model_path, map_location=lambda storage, loc: storage)
    if 'Seg' in mname:
        test_data = SegData(img_files,  transform = test_transforms, mode = 'test', **nums)
    else:
        test_data = SegmentationData(img_files,  transform = test_transforms, mode = 'test', **nums)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False, num_workers=4)
    if 'binary' in mname:
        model.binary_out = True
    else:
        model.binary_out = False
    model.eval()
    stats = gs.get_model_stats(model, test_loader)
    print('Accuracy of %s is %s'%(mname, stats['accuracy']))
    np.save('stats/%s'%mname+'.stats', stats)
    

In [4]:
mdict = {}
for mname in model_names:
    stats = np.load('stats/%s'%mname+'.stats.npy').item()
    mdict[mname] = stats

In [7]:

sens = []
spec = []
acc =  []
for mnames in model_names:
    sens.append(mdict[mnames]['sensitivity'])
    spec.append(mdict[mnames]['specificity'])
    acc.append(mdict[mnames]['accuracy'])
    
cs = {'b': (0.2980392156862745, 0.4470588235294118, 0.6901960784313725),
     'g': (0.3333333333333333, 0.6588235294117647, 0.40784313725490196),
     'r': (0.7686274509803922, 0.3058823529411765, 0.3215686274509804),
     'v': (0.5058823529411764, 0.4470588235294118, 0.6980392156862745),
     'y': (0.8, 0.7254901960784313, 0.4549019607843137),
     'c': (0.39215686274509803, 0.7098039215686275, 0.803921568627451)}    
    
plt.figure(figsize=(7, 7))    
    
diameter = 5*np.pi * (15 * np.array(acc))**2
colors = [cs['v'], cs['b'], cs['r'], cs['y'], cs['g']]
plt.scatter(sens, spec, s=diameter, c=colors, alpha=0.7)
plt.xlabel('Sensitivity', fontsize = 16)
plt.ylabel('Specificity',fontsize = 16)
#for i, mname in enumerate(model_names):
#    ax = plt.gca()
#    ax.annotate(mname, (sens[i],spec[i]))
plt.show()

    

In [ ]:
for key in mdict:
    print(key, mdict[key]['highscore_index'])
    